In [1]:
pip install seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 5.7 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Mengimpor library yang diperlukan
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# Masukan train data yang sudah di-preprocessing di variable df_train
df_train = pd.read_csv('/kaggle/input/dataset/train.csv')

# Masukan test data yang sudah di-preprocessing di variable df_test
df_test = pd.read_csv('/kaggle/input/dataset/test.csv')

In [4]:
df_train.isnull().sum()

Penumpang_id               0
Asal                     161
Sleeper_Plus             171
Kabin                    162
Tujuan                   150
Umur                     154
VIP                      158
Servis_Kamar_Tambahan    150
Belanja_Makanan          153
Belanja_Merchandise      168
Spa                      146
Bioskop                  162
Nama                     165
Tiba_Ditujuan              0
dtype: int64

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6954 entries, 0 to 6953
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Penumpang_id           6954 non-null   object 
 1   Asal                   6793 non-null   object 
 2   Sleeper_Plus           6783 non-null   object 
 3   Kabin                  6792 non-null   object 
 4   Tujuan                 6804 non-null   object 
 5   Umur                   6800 non-null   float64
 6   VIP                    6796 non-null   object 
 7   Servis_Kamar_Tambahan  6804 non-null   float64
 8   Belanja_Makanan        6801 non-null   float64
 9   Belanja_Merchandise    6786 non-null   float64
 10  Spa                    6808 non-null   float64
 11  Bioskop                6792 non-null   float64
 12  Nama                   6789 non-null   object 
 13  Tiba_Ditujuan          6954 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 713.2+ K

In [6]:
def handling_missing_data(df):
    
    # Drop kolom
    df = df.drop(columns=['Nama'], axis=1);

#     # Handling missing value numeric 
#     df['Umur'] = df['Umur'].fillna(df['Umur'].mean());
#     df['Servis_Kamar_Tambahan'] = df['Servis_Kamar_Tambahan'].fillna(df['Servis_Kamar_Tambahan'].mean());
#     df['Belanja_Makanan'] = df['Belanja_Makanan'].fillna(df['Belanja_Makanan'].mean());
#     df['Spa'] = df['Spa'].fillna(df['Spa'].mean());
#     df['Belanja_Merchandise'] = df['Belanja_Merchandise'].fillna(df['Belanja_Merchandise'].mean());
#     df['Bioskop'] = df['Bioskop'].fillna(df['Bioskop'].mean());

#     # Handling missing value object
#     df['Asal'] = df['Asal'].fillna(df['Asal'].mode()[0]);
#     df['Sleeper_Plus'] = df['Sleeper_Plus'].fillna(df['Sleeper_Plus'].mode()[0]);
#     df['Kabin'] = df['Kabin'].fillna(df['Kabin'].mode()[0]);
#     df['Tujuan'] = df['Tujuan'].fillna(df['Tujuan'].mode()[0]);
#     df['VIP'] = df['VIP'].fillna(df['VIP'].mode()[0]);
    
    return df


df_train = handling_missing_data(df_train);
df_test = handling_missing_data(df_test);

In [7]:
df_train.isnull().sum()

Penumpang_id               0
Asal                     161
Sleeper_Plus             171
Kabin                    162
Tujuan                   150
Umur                     154
VIP                      158
Servis_Kamar_Tambahan    150
Belanja_Makanan          153
Belanja_Merchandise      168
Spa                      146
Bioskop                  162
Tiba_Ditujuan              0
dtype: int64

In [8]:
df_train.head()

,Penumpang_id,Asal,Sleeper_Plus,Kabin,Tujuan,Umur,VIP,Servis_Kamar_Tambahan,Belanja_Makanan,Belanja_Merchandise,Spa,Bioskop,Tiba_Ditujuan
0,0001_01,Jakarta,False,B/0/P,Bali,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,0002_01,Lampung,False,F/0/S,Bali,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,0003_01,Jakarta,False,A/0/S,Bali,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,0003_02,Jakarta,False,A/0/S,Bali,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,0004_01,Lampung,False,F/1/S,Bali,16.0,False,303.0,70.0,151.0,565.0,2.0,True


In [9]:
# split PassengerId column into its component parts (group, and id ) as separate features  
df_train[['grup', 'id']] = df_train['Penumpang_id'].str.split(pat='_', expand=True) 
df_test[['grup', 'id']] = df_test['Penumpang_id'].str.split(pat='_', expand=True)

# split Cabin column into its component parts (deck, cabin id, side ) as separate features 
df_train[['deck', 'no_kabin', 'sisi']] = df_train.Kabin.str.split('/', expand=True) 
df_test[['deck', 'no_kabin', 'sisi']] = df_test.Kabin.str.split('/', expand=True) 

In [10]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6954 entries, 0 to 6953
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Penumpang_id           6954 non-null   object 
 1   Asal                   6793 non-null   object 
 2   Sleeper_Plus           6783 non-null   object 
 3   Kabin                  6792 non-null   object 
 4   Tujuan                 6804 non-null   object 
 5   Umur                   6800 non-null   float64
 6   VIP                    6796 non-null   object 
 7   Servis_Kamar_Tambahan  6804 non-null   float64
 8   Belanja_Makanan        6801 non-null   float64
 9   Belanja_Merchandise    6786 non-null   float64
 10  Spa                    6808 non-null   float64
 11  Bioskop                6792 non-null   float64
 12  Tiba_Ditujuan          6954 non-null   bool   
 13  grup                   6954 non-null   object 
 14  id                     6954 non-null   object 
 15  deck

In [11]:
numeric = ['Umur', 'grup', 'id', 'no_kabin', 'Servis_Kamar_Tambahan', 'Belanja_Makanan', 'Belanja_Merchandise','Spa','Bioskop',] 
categorical = ['Asal', 'Tujuan', 'deck', 'sisi', 'Sleeper_Plus', 'VIP'] 

In [12]:
# # Converting target column booleans to corresponding Integer values 
# df_train.Tiba_Ditujuan = df_train.Tiba_Ditujuan.replace({True: 1, False:0})

In [13]:
# df_train = df_train[df_train.Tiba_Ditujuan.notna()].Tiba_Ditujuan.astype(int) 

In [14]:
# # setting original PassengerId column as row index 
# df_train.set_index('Penumpang_id', inplace=True) 
# df_test.set_index('Penumpang_id', inplace=True) 

In [15]:
# converting into numeric dtypes where numbers are appearing as object datatype 
df_train[['grup', 'id', 'no_kabin']] = df_train[['grup', 'id', 'no_kabin']].apply(pd.to_numeric, axis=0)
df_test[['grup', 'id', 'no_kabin']] = df_test[['grup', 'id', 'no_kabin']].apply(pd.to_numeric, axis=0)

In [16]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6954 entries, 0 to 6953
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Penumpang_id           6954 non-null   object 
 1   Asal                   6793 non-null   object 
 2   Sleeper_Plus           6783 non-null   object 
 3   Kabin                  6792 non-null   object 
 4   Tujuan                 6804 non-null   object 
 5   Umur                   6800 non-null   float64
 6   VIP                    6796 non-null   object 
 7   Servis_Kamar_Tambahan  6804 non-null   float64
 8   Belanja_Makanan        6801 non-null   float64
 9   Belanja_Merchandise    6786 non-null   float64
 10  Spa                    6808 non-null   float64
 11  Bioskop                6792 non-null   float64
 12  Tiba_Ditujuan          6954 non-null   bool   
 13  grup                   6954 non-null   int64  
 14  id                     6954 non-null   int64  
 15  deck

In [17]:
df_train.drop(['Kabin'], axis=1, inplace=True)
df_test.drop(['Kabin'], axis=1, inplace=True)

In [18]:
df_train.info()
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6954 entries, 0 to 6953
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Penumpang_id           6954 non-null   object 
 1   Asal                   6793 non-null   object 
 2   Sleeper_Plus           6783 non-null   object 
 3   Tujuan                 6804 non-null   object 
 4   Umur                   6800 non-null   float64
 5   VIP                    6796 non-null   object 
 6   Servis_Kamar_Tambahan  6804 non-null   float64
 7   Belanja_Makanan        6801 non-null   float64
 8   Belanja_Merchandise    6786 non-null   float64
 9   Spa                    6808 non-null   float64
 10  Bioskop                6792 non-null   float64
 11  Tiba_Ditujuan          6954 non-null   bool   
 12  grup                   6954 non-null   int64  
 13  id                     6954 non-null   int64  
 14  deck                   6792 non-null   object 
 15  no_k

In [19]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan) 

df_train[categorical] = ordinal_encoder.fit_transform(df_train [categorical]) 
df_test[categorical] = ordinal_encoder.transform(df_test[categorical])

In [20]:
from sklearn.impute import KNNImputer

# Create an instance of KNNImputer
imputer = KNNImputer(n_neighbors=5)

# Apply the imputer to the numeric columns in the train dataset
df_train[numeric + categorical] = imputer.fit_transform(df_train[numeric + categorical])

# Apply the imputer to the numeric columns in the test dataset
df_test[numeric + categorical] = imputer.transform(df_test[numeric + categorical])

In [21]:
df_train

,Penumpang_id,Asal,Sleeper_Plus,Tujuan,Umur,VIP,Servis_Kamar_Tambahan,Belanja_Makanan,Belanja_Merchandise,Spa,Bioskop,Tiba_Ditujuan,grup,id,deck,no_kabin,sisi
0,0001_01,1.0,0.0,1.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,False,1.0,1.0,1.0,0.0,0.0
1,0002_01,2.0,0.0,1.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,True,2.0,1.0,5.0,0.0,1.0
2,0003_01,1.0,0.0,1.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,False,3.0,1.0,0.0,0.0,1.0
3,0003_02,1.0,0.0,1.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,False,3.0,2.0,0.0,0.0,1.0
4,0004_01,2.0,0.0,1.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,True,4.0,1.0,5.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6949,7376_01,1.0,1.0,1.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,True,7376.0,1.0,4.0,477.0,0.0
6950,7377_01,2.0,1.0,0.0,43.0,0.0,0.0,0.0,0.0,0.0,0.0,False,7377.0,1.0,6.0,1186.0,0.0
6951,7379_01,2.0,0.0,1.0,20.0,0.0,9.0,0.0,1540.0,0.0,0.0,False,7379.0,1.0,5.0,1416.0,1.0
6952,7385_01,2.0,0.0,0.0,19.0,0.0,40.0,0.0,77.0,572.0,0.0,False,7385.0,1.0,5.0,1417.0,1.0


In [22]:
from sklearn.preprocessing import MinMaxScaler

# calculate min & max values for all categorical features combined 
min_cat = df_train[categorical].min().min()
max_cat = df_train[categorical].max().max()

# Scaling using MinMaxScaler 
mmScaler = MinMaxScaler(feature_range=(min_cat, max_cat)) 
df_train[numeric] = mmScaler.fit_transform(df_train[numeric]) 
df_test[numeric] = mmScaler.transform(df_test[numeric]) 

In [23]:
df_train

,Penumpang_id,Asal,Sleeper_Plus,Tujuan,Umur,VIP,Servis_Kamar_Tambahan,Belanja_Makanan,Belanja_Merchandise,Spa,Bioskop,Tiba_Ditujuan,grup,id,deck,no_kabin,sisi
0,0001_01,1.0,0.0,1.0,3.455696,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,False,0.000000,0.0,1.0,0.000000,0.0
1,0002_01,2.0,0.0,1.0,2.126582,0.0,0.053256,0.002113,0.014282,0.171501,0.012763,True,0.000948,0.0,5.0,0.000000,1.0
2,0003_01,1.0,0.0,1.0,5.139241,1.0,0.021009,0.839634,0.000000,2.097688,0.014213,False,0.001895,0.0,0.0,0.000000,1.0
3,0003_02,1.0,0.0,1.0,2.924051,0.0,0.000000,0.301244,0.211948,1.039941,0.055981,False,0.001895,1.0,0.0,0.000000,1.0
4,0004_01,2.0,0.0,1.0,1.417722,0.0,0.148042,0.016436,0.086265,0.176499,0.000580,True,0.002843,0.0,5.0,0.004584,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6949,7376_01,1.0,1.0,1.0,2.303797,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,True,6.989575,0.0,4.0,2.186640,0.0
6950,7377_01,2.0,1.0,0.0,3.810127,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,False,6.990523,0.0,6.0,5.436804,0.0
6951,7379_01,2.0,0.0,1.0,1.772152,0.0,0.004397,0.000000,0.879785,0.000000,0.000000,False,6.992418,0.0,5.0,6.491159,1.0
6952,7385_01,2.0,0.0,0.0,1.683544,0.0,0.019544,0.000000,0.043989,0.178686,0.000000,False,6.998105,0.0,5.0,6.495743,1.0


In [24]:
# from sklearn.preprocessing import LabelEncoder

In [25]:
# le = LabelEncoder()

# train_columns = ['Asal', 'Sleeper_Plus', 'Umur', 'Tujuan', 'VIP', 'Tiba_Ditujuan', 'grup', 'id', 'deck', 'no_kabin','sisi']
# test_columns = ['Asal', 'Sleeper_Plus', 'Umur', 'Tujuan', 'VIP', , 'grup', 'id', 'deck', 'no_kabin','sisi']

# for col in train_columns:
#     df_train[col] = le.fit_transform(df_train[col])
    
# for col in test_columns:
#     df_test[col] = le.fit_transform(df_test[col])

In [26]:
df_train.head()

,Penumpang_id,Asal,Sleeper_Plus,Tujuan,Umur,VIP,Servis_Kamar_Tambahan,Belanja_Makanan,Belanja_Merchandise,Spa,Bioskop,Tiba_Ditujuan,grup,id,deck,no_kabin,sisi
0,0001_01,1.0,0.0,1.0,3.455696,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,False,0.000000,0.0,1.0,0.000000,0.0
1,0002_01,2.0,0.0,1.0,2.126582,0.0,0.053256,0.002113,0.014282,0.171501,0.012763,True,0.000948,0.0,5.0,0.000000,1.0
2,0003_01,1.0,0.0,1.0,5.139241,1.0,0.021009,0.839634,0.000000,2.097688,0.014213,False,0.001895,0.0,0.0,0.000000,1.0
3,0003_02,1.0,0.0,1.0,2.924051,0.0,0.000000,0.301244,0.211948,1.039941,0.055981,False,0.001895,1.0,0.0,0.000000,1.0
4,0004_01,2.0,0.0,1.0,1.417722,0.0,0.148042,0.016436,0.086265,0.176499,0.000580,True,0.002843,0.0,5.0,0.004584,1.0


In [27]:
# One hot encoding 

def one_hot_encoder(df):
    df = pd.get_dummies(df, columns=categorical, dtype=int, drop_first=True)
    pd.set_option('display.max_columns', None)
    
    return df

In [28]:
df_train = one_hot_encoder(df_train)
df_test = one_hot_encoder(df_test)

In [29]:
# from sklearn.preprocessing import LabelEncoder

In [30]:
# le = LabelEncoder()

# df_train['Tiba_Ditujuan']= le.fit_transform(df_train['Tiba_Ditujuan']) 

In [31]:
df_train.columns

Index(['Penumpang_id', 'Umur', 'Servis_Kamar_Tambahan', 'Belanja_Makanan',
       'Belanja_Merchandise', 'Spa', 'Bioskop', 'Tiba_Ditujuan', 'grup', 'id',
       'no_kabin', 'Asal_0.2', 'Asal_0.4', 'Asal_0.8', 'Asal_1.0', 'Asal_1.2',
       'Asal_1.4', 'Asal_1.6', 'Asal_1.8', 'Asal_2.0', 'Tujuan_0.2',
       'Tujuan_0.6', 'Tujuan_0.8', 'Tujuan_1.0', 'Tujuan_1.2', 'Tujuan_1.4',
       'Tujuan_1.6', 'Tujuan_1.8', 'Tujuan_2.0', 'deck_0.6', 'deck_1.0',
       'deck_2.0', 'deck_2.2', 'deck_2.4', 'deck_2.6', 'deck_2.8', 'deck_3.0',
       'deck_3.2', 'deck_3.4', 'deck_3.6', 'deck_3.8', 'deck_4.0', 'deck_4.2',
       'deck_4.4', 'deck_4.6', 'deck_4.8', 'deck_5.0', 'deck_5.2', 'deck_5.4',
       'deck_5.6', 'deck_5.8', 'deck_6.0', 'deck_7.0', 'sisi_0.2', 'sisi_0.4',
       'sisi_0.6', 'sisi_0.8', 'sisi_1.0', 'Sleeper_Plus_0.2',
       'Sleeper_Plus_0.4', 'Sleeper_Plus_0.6', 'Sleeper_Plus_0.8',
       'Sleeper_Plus_1.0', 'VIP_0.2', 'VIP_0.4', 'VIP_0.6', 'VIP_1.0'],
      dtype='object')

In [32]:
df_test.columns

Index(['Penumpang_id', 'Umur', 'Servis_Kamar_Tambahan', 'Belanja_Makanan',
       'Belanja_Merchandise', 'Spa', 'Bioskop', 'grup', 'id', 'no_kabin',
       'Asal_0.4', 'Asal_0.8', 'Asal_1.0', 'Asal_1.2', 'Asal_1.6', 'Asal_1.8',
       'Asal_2.0', 'Tujuan_0.6', 'Tujuan_0.8', 'Tujuan_1.0', 'Tujuan_1.2',
       'Tujuan_1.4', 'Tujuan_1.6', 'Tujuan_1.8', 'Tujuan_2.0', 'deck_1.0',
       'deck_2.0', 'deck_2.6', 'deck_3.0', 'deck_4.0', 'deck_4.2', 'deck_4.6',
       'deck_4.8', 'deck_5.0', 'deck_5.4', 'deck_5.6', 'deck_6.0', 'sisi_0.2',
       'sisi_0.4', 'sisi_0.6', 'sisi_0.8', 'sisi_1.0', 'Sleeper_Plus_0.2',
       'Sleeper_Plus_0.4', 'Sleeper_Plus_0.6', 'Sleeper_Plus_0.8',
       'Sleeper_Plus_1.0', 'VIP_1.0'],
      dtype='object')

In [33]:
# Dapatkan daftar fitur dari df_train dan df_test
fitur_train = set(df_train.columns)
fitur_test = set(df_test.columns)

# Temukan perbedaan dalam set fitur
fitur_hilang_di_test = fitur_train - fitur_test

# Untuk fitur yang ada di data pelatihan tetapi hilang di data pengujian, tambahkan kolom dengan nilai 0.
for fitur in fitur_hilang_di_test:
    df_test[fitur] = 0

# Urutkan kolom di df_test agar sesuai dengan df_train.
df_test = df_test[df_train.columns]


In [34]:
X = df_train.drop(columns=['Tiba_Ditujuan'])
y = df_train['Tiba_Ditujuan']

In [35]:
# Memisahkan data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Membuat model RandomForestClassifier
rf = RandomForestClassifier()

# Menentukan hyperparameters yang ingin diuji
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Membuat objek GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1)

# Melatih model dengan data latih
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
398 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/site-packages/sklearn/base.py", line 1144, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.8/site-packages/sklearn/base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.8/site-packages/sklearn/utils/_p

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [10, 20, 30, None],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]})

In [36]:
if 'Tiba_Ditujuan' in df_test.columns:
    df_test = df_test.drop('Tiba_Ditujuan', axis=1)

In [37]:
y_result = grid_search.predict(df_test)

In [38]:
# print(df_test.columns)

In [39]:
df_submission = pd.DataFrame({'Penumpang_id': df_test['Penumpang_id']})
df_submission['Tiba_Ditujuan'] = y_result

In [40]:
from sklearn.metrics import classification_report

# Predict the labels for the test data
y_pred = grid_search.predict(X_test)

# Generate and print the classification report
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

       False       0.77      0.85      0.81       695
        True       0.83      0.75      0.79       696

    accuracy                           0.80      1391
   macro avg       0.80      0.80      0.80      1391
weighted avg       0.80      0.80      0.80      1391



In [41]:
from sklearn.metrics import accuracy_score

# Mengambil model terbaik yang ditemukan oleh GridSearchCV
best_model = grid_search.best_estimator_

# Memprediksi label data uji menggunakan model terbaik
y_pred = best_model.predict(X_test)

# Menghitung akurasi berdasarkan prediksi yang dibuat oleh model terhadap data uji sebenarnya
accuracy = accuracy_score(y_test, y_pred)

print(f"Akurasi dari model terbaik adalah: {accuracy * 100:.4f}%")


Akurasi dari model terbaik adalah: 80.1582%


In [42]:
df_submission.to_csv('submission.csv', index=False)